# Introduction to NeXL
Nicholas W. M. Ritchie 30-Apr-2023 (nicholas.ritchie@nist.gov)

## What is NeXL?
NeXL is a collection of packages (libraries) for the [Julia language](https://julialang.org) that implement common X-ray microanalysis algorithms, physical data and utilities.

The core NeXL packages are
* [NeXLCore](https://github.com/usgovnist/NeXLCore) - Defines core concepts like elements, X-rays, atomic shells, k-ratios and fundamental electron and X-ray algorithms. 
* [NeXLMatrixCorrection](https://github.com/usgovnist/NeXLMatrixCorrection) - Implements algorithms for performing matrix correction for bulk samples
* [NeXLSpectrum](https://github.com/usgovnist/NeXLSpectrum) - Implements data types and algorithms to handle spectra and hyper-spectra

NeXL builds on many packages from the Julia infrastructure including
* [Gadfly](https://github.com/GiovineItalia/Gadfly.org) - A "grammar of graphics" plotting package
* [DataFrames](https://github.com/JuliaData/DataFrames.jl) - "Pandas-like" tools for working with tabular data

## Why Julia?
Julia is a high-performance scripting language designed for numerical data analysis.  The base Julia libraries provide basic scalar, vector and matrix numerical analysis tools.  Julia differs from most scriptable languages in that all code is compiled before execution.  This leads to a "slow then fast" user experience.  The first time a method is used, it and all the methods it depends upon must be compiled.  This can lead to a significant delay.  However, once compiled, the code is fast - often as fast as C or Fortran code.  You will notice this "slow then fast" nature as you work with this notebook.  It is particularly evident when creating plots.

## NeXLCore
We will start by exploring NeXLCore since this package provides core functionality for the other NeXL packages.

In [ ]:
using DrWatson
@quickactivate "IUMAS"

using NeXLCore

### Elements
We will first construct a datum representing an element and then manipulate this object to extract elemental data.

Julia is not an object-oriented language.  Instead it uses what is called "multiple dispatch" to determine which method implementation to call dependent on the arguments handed to the function.

In [ ]:
el = n"Tc"  # Here we are using a special syntactic sugar to convert the string "Tc" into an Element struct

Let's get some properties of this elemental datum.

In [ ]:
z(el), a(el), symbol(el), name(el)

We can combine elements into materials.  The `mat"..."` macro converts expressions into `Material` structs.

In [ ]:
mat = mat"Ca(PO4)3OH"
mat, typeof(mat)

An alternative syntax allows you to specify additional properties of the `Material`.

In [ ]:
mat = parse(Material, "Ca(PO4)3OH", name = "Apatite", density = 2.2)

In addition to being able to parse chemical formulae, NeXLCore can also parse expressions like the next one.  In this case, the multiplier represents the mass fraction of the element.  The right hand side can also be a chemical formula as though the material was made from various mass fractions of consituent materials.

In [ ]:
adm = parse(Material, "0.3399*O+0.0664*Al+0.0405*Si+0.0683*Ca+0.0713*Ti+0.1055*Zn+0.3037*Ge", name="ADM-6005a")

In addition to elements and materials, `NeXLCore` provides mechanisms to work with characteristic X-rays, atomic shells and other atomic physics data.  The `n"..."` macro can be used to create `Element`, `CharXRay`, `AtomicSubShell` and `SubShell` structs.

In [ ]:
cxr = n"Tc K-L3"
cxr, typeof(cxr)

In [ ]:
ass = n"Tc K"
ass, typeof(ass)

Let's get some properties of the `CharXRay` datum.  You'll notice that the energy is in electron-volts as is the case for all energies within NeXL.

In [ ]:
energy(cxr), inner(cxr), energy(inner(cxr)), outer(cxr), energy(outer(cxr)), typeof(inner(cxr))

To determine every characteristic X-ray generated by a particular element, you can use the `characteristic(...)`. To generate sub-sets of the transitions replace `alltransitions` with `ktransitions`, `ltransitions` or `mtransitions`.

In [ ]:
cxrs=characteristic(el, alltransitions)
cxrs, length(cxrs)

Now we can use various functions to extract X-ray related physical data from elements and materials.  The `mac(...)` function returns the mass absorption coeffficient in $cm^2/g$.

In [ ]:
mac(mat, n"Ca K-L3"), mac(n"Ca", n"Ca K-L3"), mac(mat, 3692.0), mac(n"Ca", 3692.0)

This is an example of one of Julia's core design features - multiple dispatch.  The function `mac(...)` is called on four distinct argument types.
* `mac(Material, CharXRay)`
* `mac(Element, CharXRay)`
* `mac(Material, Float64)`
* `mac(Element, Float64)`

This is similar but extends on the way object oriented code can use the object type to determine which function implementation to call.
 

`NeXL` uses the `Gadfly` library to plot spectra and other data items.

In [ ]:
using Gadfly
plot(e->mac(mat, e), 100.0, 1.0e4, Scale.y_log10)

In [ ]:
plot(
    layer(e->mac(mat, e), 100.0, 1.0e4, Theme(default_color="Red")),
    layer(e->mac(n"Ca", e), 100.0, 1.0e4, Theme(default_color="Green")),
    layer(e->mac(n"P", e), 100.0, 1.0e4, Theme(default_color="Blue")),
    Scale.y_log10
)

There is much more functionality in NeXLCore, but that is enought to get us started.  Next let's take a look at NeXLSpectrum.

## NeXLSpectrum

NeXLSpectrum defines methods for reading, writing, manipulating, plotting and fitting X-ray spectra and X-ray spectrum images ("hyperspectra"). 

In [ ]:
using NeXLSpectrum

First, we will read a spectrum from disk using the `loadspectrum(...)` method.  It take a filename and is able to sniff the file type for many common spectrum file formats.

In [ ]:
sp = loadspectrum(joinpath(datadir(), "exp_raw", "ADM6005a spectra","ADM-6005a_1.msa"))

In [ ]:
set_default_plot_size(10inch, 4inch)
spec_files = [ joinpath(datadir(), "exp_raw", "ADM6005a spectra","ADM-6005a_$i.msa") for i in 1:15 ]
specs=loadspectrum.(spec_files)
plot(specs..., klms=[n"O", n"Al", n"Si", n"Ca", n"Ti", n"Zn", n"Ge"])

In [ ]:
plot(specs..., klms=[n"O", n"Al", n"Si", n"Ca", n"Ti", n"Zn", n"Ge"], xmax=6.0e3)

To quantify this data, we need standard spectra.  The standard spectra are preprocessed to optimize the fitting process.  The spectrum files are read from disk and must contain the following properties `:BeamEnergy`, `:LiveTime`, `:ProbeCurrent` and `:TakeOffAngle` to be fit and quantified.  It is possible to pre-load the standard spectra and edit the properties in the script if they are not present in the spectrum file.

In [ ]:
path=joinpath(datadir(), "exp_raw", "ADM6005a spectra")
refs=references( [
        reference(n"O", joinpath(path, "SiO2 std.msa"), mat"SiO2"),
        reference(n"Si", joinpath(path, "SiO2 std.msa"), mat"SiO2"),
        reference(n"Al", joinpath(path, "Al std.msa"), mat"Al"),
        reference(n"Ca", joinpath(path, "CaF2 std.msa"), mat"CaF2"),
        reference(n"Ti", joinpath(path, "Ti trimmed.msa"), mat"Ti"),
        reference(n"Zn", joinpath(path, "Zn std.msa"), mat"Zn"),
        reference(n"Ge", joinpath(path, "Ge std.msa"), mat"Ge"),
    ], 132.0
)
using DataFrames
asa(DataFrame, refs)  # Describe the references in a DataFrame

The references are then used to fit the unknown spectra and matrix correction is performed.  The `quantify(...)` method can be used to perform both operations or the `fit_spectrum(...)` method can be used to fit the references to the unknowns to produce k-ratios and then the k-ratios fed to the `quantify(...)` method to perform matrix correction.  The later two-step process is useful when you want to access the k-ratio data.

In [ ]:
qrs=map(sp->quantify(sp, refs),specs)
qdf=asa(DataFrame, qrs, nominal=adm)

We can use the `describe(...)` method from `DataFrames` to generate summary statistics.

In [ ]:
describe(qdf[1:end-1,2:end], :mean, :std, :min, :max)

Or we can take a closer look at the data from one spectrum.

In [ ]:
asa(DataFrame, qrs[1])

Associated with each spectrum are a collection of properties.  These properties are often read from the spectrum file but can also be assigned manually.

In [ ]:
NeXLCore.properties(sp)

Properties are identified with a `Symbol` which is represented by a colon ':' plus a name.  There are many predefined spectrum properties but you can also define your own to associate custom data with spectra.

In [ ]:
sp[:BeamEnergy], sp[:ProbeCurrent], sp[:LiveTime], sp[:TakeOffAngle], dose(sp)

This is one way to index spectra to extract data from the spectrum.  It is also possible to extract channel count data using various different types as indices.  To demonstrate this, we will use `Integer`, `AbstractFloat` or `CharXRay` types to index the same channel in the spectrum - the channel associated with the Ca K-L3 peak.

In [ ]:
channel(n"Ca K-L3", sp), channel(Float64, n"Ca K-L3", sp), channel(energy(n"Ca K-L3"), sp), channel(Float64, energy(n"Ca K-L3"), sp)

In the first case, we index the channel directly by channel index.  In the second case, we use a floating point number which is assumed to represent an energy to index the same channel. Finally, we use a `CharXRay` to index the same channel.

In [ ]:
sp[833], sp[energy(n"Ca K-L3")], sp[n"Ca K-L3"]

Often it is useful to be able to determine which of a collection of spectra collected from a single material under similar conditions are similar to one another.  The notion being that the 'best' spectra are those that are most similar.  These can then be summed together to produce a single spectrum that best represents the material.  The `findsimilar(...)` method extracts the similar spectra from a list and removes outliers.

In [ ]:
plot(sum(findsimilar(specs, refs.detector, n"O")), klms=[n"O", n"Al", n"Si", n"Ca", n"Ti", n"Zn", n"Ge"])

The similarity is determined by comparing each spectrum with the sum of the other spectra and calculating a score that respects count statistics.  Similar spectra have a `similarity(...)` score of approximately unity.

In [ ]:
DataFrame(Name=name.(specs), S1=NeXLSpectrum.similarity(specs, refs.detector, adm), S2=NeXLSpectrum.similarity(specs), Counts=integrate.(specs))

#### HyperSpectra with NeXLSpectrum

A `HyperSpectrum` represents a multi-dimensional array of point spectra.  `NeXLSpectrum` can handle a arbitrary number of dimensions like 1-D (linescans), 2-D (area scans) or higher dimensions.

There are currently three ways to load a hyperspectrum.
* From a Lispix-style RPL/RAW file pair
* From a SEMantics-style PTZ file
* From a HyperSpy-style HDF5 file

We will use the `DataDeps` library to download the hyperspectrum data on demand from the NIST data website.  You will need to be connected to the Internet.

In [ ]:
# DataDeps is a utility for downloading data on demand from a URL
using DataDeps

# Where do I want to put the data?
ENV["DATADEPS_LOAD_PATH"] = joinpath(datadir(), "exp_raw")
ENV["DATADEPS_ALWAYS_ACCEPT"]="true"

# Register the data sets using the names "MnDeepSeaNodule" and "MnDeepSeaNoduleStandards"
register(DataDep("MnDeepSeaNodule",
    """
    Dataset:       Deep Sea Manganese Nodule
    Acquired by:   Nicholas W.M. Ritchie
    License:       CC-SA 3.0
    """,
    raw"https://data.nist.gov/od/ds/mds2-2467/MnNodule.tar.gz",
    "5b5b6623b8f4daca3ff3073708442ac5702ff690aa12668659875ec5642b458d",
    post_fetch_method=DataDeps.unpack
));

register(DataDep("MnDeepSeaNoduleStandards",
    """
    Dataset:       Standard Spectra for Deep Sea Manganese Nodule Dataset
    Acquired by:   Nicholas W.M. Ritchie
    License:       CC-SA 3.0
    """,
    raw"https://data.nist.gov/od/ds/mds2-2467/MnNodule_Standards.tar.gz",
    "69283ba72146932ba451e679cf02fbd6b350f96f6d012d50f589ed9dd2e35f1a",
    post_fetch_method=DataDeps.unpack
));

The data is in RPL/RAW format - a format designed by David Bright at NIST and available from many EDS vendor's software.  In addition to the data in the RPL/RAW format, we also need to provide spectrum meta-data and the detector energy scale.

In [ ]:
raw = readrplraw(joinpath(datadep"MnDeepSeaNodule","map[15]"))
les = LinearEnergyScale(0.0, 10.0)
props = Dict{Symbol,Any}(
    :LiveTime => 0.72*4.0*18.0*3600.0/(1024.0*1024.0),
    :BeamEnergy => 20.0e3,
    :TakeOffAngle => deg2rad(35.0),
    :ProbeCurrent => 1.0,
    :Name => "Deep Sea Mn Nodule",
)
hs = HyperSpectrum(les, props, raw, fov = (0.2, 0.2))

It is easy to generate crude elemental maps from `HyperSpectrum` structs.  These are simple ROI maps constructed by summing a range of channels around the central channel for the specified X-ray transition.

In [ ]:
hs[n"Mn K-L3"]

Or I can create a RGB-colorized image where R represents an ROI around the first chararacteristic X-ray, green the second and blue the third.

In [ ]:
hs[n"Mn K-L3", n"Si K-L3", n"O K-L3"]

Much like spectra, hyperspectra have properties.  However, the same set of properties apply to all spectra within a hyperspectrum. There is one exception, `:LiveTime`, which can be an array of the same dimension as the hyperspectrum to represent a unique live-time per pixel.

In [ ]:
NeXLCore.properties(hs)

To speed up processing bin the data using a block size > 1. 4 will provide 16x faster processing but lower resolution results.

In [ ]:
# This speeds up the processing by binning a 1024 x 1024 hyperspectrum to 1024/block_size x 1024/block_size.
block_size = 1
hs = block_size > 1 ? block(hs, (block_size, block_size)) : hs

Unfortunately, ROI maps suffer from many shortcomings including
* not being background corrected to eliminate the continuum signal
* not being scaled for differences in generation efficiency between different characteristic X-rays.

While they are convenient as a first glimse at the data, they can be very deceptive and should rarely be included in reports.

Instead, it is better to fit the point spectra within the hyperspectrum using measured references.

In [ ]:
hs_elms = [ n"Ag", n"Al", n"Ba", n"C", n"Ca", n"Ce", n"Cr", n"Cu", n"Fe", n"S", n"P", n"K", n"Mg", n"O", n"Mn", n"Na", n"Cl", n"Ni", n"Si", n"Ti", n"Zn" ]
plot(maxpixel(hs), klms = hs_elms, xmax=10.0e3)

In [ ]:
print(symbol.(sort(hs_elms)))

In [ ]:
refs = references( [
        reference(n"C", joinpath(datadep"MnDeepSeaNoduleStandards", "C std.msa")),
        reference(n"O", joinpath(datadep"MnDeepSeaNoduleStandards", "MgO std.msa")),
        reference(n"Na", joinpath(datadep"MnDeepSeaNoduleStandards", "NaCl std.msa")),
        reference(n"Mg", joinpath(datadep"MnDeepSeaNoduleStandards", "Mg std.msa")),
        reference(n"Al", joinpath(datadep"MnDeepSeaNoduleStandards", "Al std.msa")),
        reference(n"Si", joinpath(datadep"MnDeepSeaNoduleStandards", "Si std.msa")),
        reference(n"P", joinpath(datadep"MnDeepSeaNoduleStandards", "GaP std.msa")), 
        reference(n"S", joinpath(datadep"MnDeepSeaNoduleStandards", "FeS2 std.msa")), 
        reference(n"Cl", joinpath(datadep"MnDeepSeaNoduleStandards", "NaCl std.msa")), 
        reference(n"K", joinpath(datadep"MnDeepSeaNoduleStandards", "KBr std.msa")), 
        reference(n"Ca", joinpath(datadep"MnDeepSeaNoduleStandards", "CaF2 std.msa")), 
        reference(n"Ti", joinpath(datadep"MnDeepSeaNoduleStandards", "Ti std.msa")), 
        reference(n"Cr", joinpath(datadep"MnDeepSeaNoduleStandards", "Cr std.msa")), 
        reference(n"Mn", joinpath(datadep"MnDeepSeaNoduleStandards", "Mn std.msa")), 
        reference(n"Fe", joinpath(datadep"MnDeepSeaNoduleStandards", "Fe std.msa")), 
        reference(n"Ni", joinpath(datadep"MnDeepSeaNoduleStandards", "Ni std.msa")), 
        reference(n"Cu", joinpath(datadep"MnDeepSeaNoduleStandards", "Cu std.msa")), 
        reference(n"Zn", joinpath(datadep"MnDeepSeaNoduleStandards", "Zn std.msa")), 
        reference(n"Ag", joinpath(datadep"MnDeepSeaNoduleStandards", "Ag std.msa")), 
        reference(n"Ba", joinpath(datadep"MnDeepSeaNoduleStandards", "BaF2 std.msa"))     
    ], 132.0)

asa(DataFrame, refs)

Now we will fit the references to the hyperspectrum unknown.  This is simple enough:

`krs = fit_spectrum(hs, refs, mode = :Fast)`


This is time consuming so once we've done it, we want to store the result in a file and reload the result rather than recalculate it.  `DrWatsons` provides a mechanism to do this based on a function `produce_or_load(....)`.  You specify where to put the data and it builds a filename based on the `prefix`, `suffix` and the configuration paramters in `@dict(mode, x_dim, y_dim)`.  The `suffix` specifies that the data is written using the `JLD2` library in HDF5 format.

In [ ]:
mode = :Full  # Single threaded took 92 seconds for :Fast or 6000 seconds for :Full (many cores take about 1/6 this.)

# krs = @time fit_spectrum(hs, refs, mode = mode)  

using FileIO, Parameters
x_dim, y_dim = size(hs)
data, file = produce_or_load(
        joinpath(datadir(),"exp_pro"), # Path
        @dict(mode, x_dim, y_dim),     # Configuration
        prefix="kratios", suffix="jld2" # Filename parameters
    ) do config
        Dict("kratios" => @time fit_spectrum(hs, refs, mode = mode, sigma=3.0)) # Zero k-ratios less that 3σ
end
@show file
krs = data["kratios"]

The result of `fit_spectrum(...)` is a set k-ratio maps. Which can be readily visualized and represent a more quantitative perspective on the spectrum data. The principle advantage being that the data is background corrected so that continuum is not mistaken for a small quantity of an element.

To visualize the k-ratios, it is best to pick one k-ratio per element using `optimizeks(...)` and then normalize the k-ratios on a point-by-point basis using `normalizek(...)`.  This ensures that all the k-ratios are between 0 and 1.  The function `asa(ElementalMap, ...)` facilitates this process by converting a `Vector{KRatios}` into a dictionary of images indexed by `Element` instances.

In [ ]:
using Colors
imgs = asa(ElementalMap, krs, Gray)
imgs[n"Mn"]

We can perform matrix correction on the `Vector{KRatios}` using the `quantify(...)` method.  It returns a `Materials` struct which is a memory efficient way of representing a multi-dimensional array of `Material`.

In [ ]:
# quant=quantify(krs, name=hs[:Name], ty=Float32) # Takes ~80 minute for 1024 x 1024 (~5 ms per pixel)

data, file = produce_or_load(
        joinpath(datadir(),"exp_pro"), 
        @dict(mode, x_dim, y_dim),
        prefix="quantify", suffix="jld2"
    ) do config
        Dict("mass_fractions" => @time quantify(krs, name=hs[:Name], ty=Float32))
end
@show file
quant = data["mass_fractions"]

The `Materials` struct can be indexed at a specific coordinate to extract the composition at that coordinate as a `Material`.

In [ ]:
ci = CartesianIndex(64 ÷ block_size,192 ÷ block_size)
plot(hs[ci], klms=quant[ci], xmax=10.0e3)

In [ ]:
quant[ci]

Or we can index the results using an `Element` in which case, we pull out a slice representing the mass fractions associated with that element.

In [ ]:
t=quant[n"Mn"]

Using broadcast, we can apply `Material` functions to `Materials` objects.  The results is an Array with the same shape but with the result contents.

In [ ]:
anal_tot=analyticaltotal.(quant)

In [ ]:
using Statistics
mean(anal_tot), std(anal_tot)

To convert the mass-fractions to images, we need to ensure all mass-fractions are on the range [0,1].  We can do this by normalizing the individual points to an analytical total of unity.

In [ ]:
norm_quant=asnormalized(quant)

Now let's visualize this.  In Julia, it is trivial to convert matrices representing values on the range [0,1] to images.

We will extract the plane of data corresponding to an element by indexing it with an `Element`.  `Log3Band` is a function that converts numbers between 0 and 1 to RGB values.  The mapping is displayed below in the legend.

In [ ]:
Log3Band.(norm_quant[n"Ni"])

In [ ]:
loadlegend("Log3BandBright.png")

The Ni elemental map can be seen to represent primarily values ranging from a few percent down to zero.  Albeit, this spectrum image took 18 hours to collect but the depth of information might surprise many who are used to thinking of spectrum images as a crude tool.

Let's output various perspectives from each element to the 'plots' directory.  The `Gray` function converts values on the range [0,1] to gray-scale values.

In [ ]:
# Fast k-ratio maps
imgs = asa(ElementalMap, krs, Log3Band) 
for (el, img) in imgs
    save(joinpath(plotsdir(),"$(symbol(el)) - $mode Log3Band.png"), img)
end
# Full k-ratio maps
for (el, img) in asa(ElementalMap, krs, Gray) 
    save(joinpath(plotsdir(),"$(symbol(el)) - $mode Gray.png"), img)
end
# Full quantitative maps - Fast fit
for el in keys(norm_quant)
    pl = norm_quant[el]
    save(joinpath(plotsdir(),"$(symbol(el)) - Quant $mode Gray.png"), Gray.(pl))
    save(joinpath(plotsdir(),"$(symbol(el)) - Quant $mode Log3Band.png"), Log3Band.(pl))
end

Let's compare the results from the full fit with the results from the fast fit.

In [ ]:
els = sort([ keys(imgs)...])
print(symbol.(els))

Plot the k-ratio images.

In [ ]:
map( el->imgs[el], els)

Plot the quantitative maps.

In [ ]:
map(el->Log3Band.(norm_quant[el]), els)

Clearly, the light element maps are where we see the influence of matrix correction.  When X-ray absorption is small, the k-ratio is generally a good approximation of the mass-fraction.  However, for low energy X-rays, the absorption can be strong leading to a significant underestimation of the mass fraction.  We see this particularly in the carbon data.

To explore the data, let's plot histograms of the mass fractions from each element.

In [ ]:
colors = distinguishable_colors(length(els), colorant"light blue", lchoices=range(50, stop=100, length=15))
    
plot(
    ( layer(x=quant[el], Geom.histogram, Theme(default_color=colors[i]), alpha=[0.6]) for (i, el) in enumerate(els) )...,
    Guide.xlabel("Mass Fraction"), Guide.ylabel("Counts"), 
    Guide.manual_color_key("Element", symbol.(els), colors),
    Coord.cartesian(xmin=0.0, xmax=1.0, ymax=16.0e4/(block_size^2))
)


In [ ]:
plot(
    ( layer(x=quant[el], Geom.histogram, Theme(default_color=colors[i]), alpha=[0.6]) for (i, el) in enumerate(els) )...,
    Guide.xlabel("Mass Fraction"), Guide.ylabel("Counts"), 
    Guide.manual_color_key("Element", symbol.(els), colors),
    Coord.cartesian(xmin=0.0, xmax=0.05, ymax=16.0e4/block_size^2)
)

We can easily query the quantified data using broadcast syntax to create BitMatrix based on a boolean comparison.
To determine which pixels have a mass-fraction of Si > 5% we create a mask.  In this `BitMatrix` mask, 1 represents a true comparison and 0 a false comparison.

In [ ]:
mask_si = quant[n"Si"] .> 0.05

Using similar syntax we can determine how many pixels have at least 50%, 10%, 5% and 1% of an element?

In [ ]:
DataFrame(
    Element=symbol.(els), 
    P50=map(el->count(quant[el] .> 0.5),els), 
    P10=map(el->count(quant[el] .> 0.1),els), 
    P5=map(el->count(quant[el] .> 0.05),els), 
    P1=map(el->count(quant[el] .> 0.01),els)
)

A mask can then be applied to extract and sum the requisite spectra within the hyperspectrum.

In [ ]:
plot( sum(hs, quant[n"Si"] .> 0.05), klms=els, xmax=10.0e3)

Let's do something similar for Cu > 10%.

In [ ]:
plot( sum(hs, quant[n"Cu"] .> 0.1), klms=els, xmax=10.0e3)

And silver...

In [ ]:
plot( sum(hs, quant[n"Ag"] .> 0.05), klms=els, xmax=10.0e3)

Finally, to visualize the relative abundances and locations of the elements in the quantified data, we can construct a RGB image. 

In [ ]:
using ImageCore
colorview(RGB, norm_quant[n"Mn"], norm_quant[n"Si"], norm_quant[n"O"])

Compare this with the ROI map from earlier.

In [ ]:
hs[n"Mn K-L3", n"Si K-L3", n"O K-L3"]

This is just a flavor of what you can do with spectra and hyperspectra using Julia, the Julia library infrastructure including the `NeXL` libraries.  There are many machine learning, chemometric, image analysis and other libraries that can be readily applied.